In [3]:
import sys
import idx2numpy
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
import sys
import numpy as np
import pandas as pd
import torch

def one_hot(label):
    # 用于把标签变成one-hot格式
    m = len(label)
    label_one_hot = torch.zeros([m,10])
    for i in range(m):
        label_one_hot[i,label[i]] = 1
    return label_one_hot

def preprocess(image):
    image_new = image/255
    return image_new

def rand_indices():
    indices = torch.randperm(20000)
    train_size, validation_size = 5000, 5000

    train_indices = indices[:train_size]
    validation_indices = indices[train_size:(train_size + validation_size)]
    test_indices = indices[(train_size + validation_size):]
    return train_indices, validation_indices, test_indices

def pollute_label(label):
    indices = torch.randperm(5000)
    polluted_indices = indices[:2500]
    for i in polluted_indices:
        label[i] = torch.zeros(10).scatter_(0, torch.randint(10, (1,)), 1)
    return label

def load_data_from_csv(flag):
    N = 20000
    data = pd.read_csv('mnist/train.csv').to_numpy()
    data = torch.from_numpy(data)
    image_total = data[1:N+1,1:]
    label_total = one_hot(data[:,0])

    tr_ind, val_ind, test_ind = rand_indices()
    image_train = preprocess(image_total[tr_ind])
    image_validation = preprocess(image_total[val_ind])
    image_test = preprocess(image_total[test_ind])

    if flag==1:
        label_train = pollute_label(label_total[tr_ind]).float()
    if flag==0:
        label_train = label_total[tr_ind].float()
    label_validation = label_total[val_ind].float()
    label_test = label_total[test_ind].float()
    return (image_train, label_train), (image_validation, label_validation), (image_test, label_test)


In [5]:
data_tr,data_val,data_test = load_data_from_csv(1)

In [8]:
from torch.utils.data import DataLoader, TensorDataset

# 创建数据集和数据加载器
img_tr, lab_tr = data_tr
train_dataset = TensorDataset(img_tr)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

with torch.no_grad():
    for val_img, val_label in DataLoader(TensorDataset(img_val, lab_val), batch_size=64):
        val_output = net(val_img)
        val_loss += loss_fun2(val_output, val_label).item()